In [1]:
import os
import random
import shutil

def create_long_tail_split(dataset_dir, output_dir, test_ratio=0.2, val_ratio=0.1, imbalance_factor=10, seed=42):
    """
    dataset_dir: 原始数据集文件夹，每个类别放在单独的子文件夹中
    output_dir: 输出的长尾数据集文件夹，将生成 train、val、test 三个子文件夹
    test_ratio: 测试集比例
    val_ratio: 验证集比例
    imbalance_factor: 最高类别与最低类别的样本数量比率，用于计算训练集的长尾分布
    seed: 固定随机种子，确保每次运行结果一致
    """
    # 固定随机种子
    random.seed(42)
    
    # 获取类别列表（假设每个子文件夹代表一个类别）
    classes = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    classes.sort()
    
    # 定义常见的图片扩展名
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    class_counts = {}
    for cls in classes:
        cls_path = os.path.join(dataset_dir, cls)
        images = [f for f in os.listdir(cls_path) if os.path.splitext(f)[1].lower() in image_extensions]
        class_counts[cls] = len(images)
    
    # 确定样本最多的类别数量
    max_count = max(class_counts.values())
    
    # 按原始图片数量降序排列类别，便于计算长尾采样数量
    sorted_classes = sorted(classes, key=lambda x: class_counts[x], reverse=True)
    num_classes = len(sorted_classes)
    
    # 计算训练集期望保留的样本数量（长尾分布公式，仅针对训练集部分）
    desired_train_counts = {}
    for rank, cls in enumerate(sorted_classes):
        desired = int(max_count * (1 / imbalance_factor) ** (rank / (num_classes - 1)))
        desired_train_counts[cls] = max(desired, 1)  # 确保至少保留1张
    
    # 创建输出文件夹：train、val、test 三个子文件夹，每个类别一个子文件夹
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(test_dir, cls), exist_ok=True)
    
    # 记录各类别划分后的数量
    final_counts = {}
    
    # 针对每个类别进行数据划分
    for cls in classes:
        cls_path = os.path.join(dataset_dir, cls)
        images = [f for f in os.listdir(cls_path) if os.path.splitext(f)[1].lower() in image_extensions]
        random.shuffle(images)
        
        total = len(images)
        num_test = int(total * test_ratio)
        num_val = int(total * val_ratio)
        
        # 划分 test、val 与 train
        test_images = images[:num_test]
        val_images = images[num_test:num_test+num_val]
        train_images = images[num_test+num_val:]
        
        # 对训练集进行长尾采样：如果训练集数量大于期望数量，则随机采样 desired_train_counts[cls] 个样本
        desired_num = desired_train_counts[cls]
        if len(train_images) > desired_num:
            train_images = random.sample(train_images, desired_num)
        
        # 将各个集合的图片复制到相应文件夹中
        for img in train_images:
            src = os.path.join(cls_path, img)
            dst = os.path.join(train_dir, cls, img)
            shutil.copy(src, dst)
        for img in val_images:
            src = os.path.join(cls_path, img)
            dst = os.path.join(val_dir, cls, img)
            shutil.copy(src, dst)
        for img in test_images:
            src = os.path.join(cls_path, img)
            dst = os.path.join(test_dir, cls, img)
            shutil.copy(src, dst)
        
        final_counts[cls] = {
            'original': total,
            'train': len(train_images),
            'val': len(val_images),
            'test': len(test_images)
        }
        print(f'类别 {cls}: 原始 {total} 张, 训练集 {len(train_images)} 张, 验证集 {len(val_images)} 张, 测试集 {len(test_images)} 张')
    
    # 输出所有类别的划分统计信息
    print("\n划分后的类别数量统计：")
    for cls, counts in final_counts.items():
        print(f"{cls}: 训练集 {counts['train']} 张, 验证集 {counts['val']} 张, 测试集 {counts['test']} 张")
    
if __name__ == "__main__":
    # 替换为你实际数据集所在路径和目标输出路径
    dataset_dir = '/Users/yaogunzhishen/Desktop/Aerial_Landscapes'
    output_dir = '/Users/yaogunzhishen/Desktop/9517data'
    create_long_tail_split(dataset_dir, output_dir, test_ratio=0.2, val_ratio=0.1, imbalance_factor=10, seed=42)

类别 Agriculture: 原始 800 张, 训练集 560 张, 验证集 80 张, 测试集 160 张
类别 Airport: 原始 800 张, 训练集 560 张, 验证集 80 张, 测试集 160 张
类别 Beach: 原始 800 张, 训练集 560 张, 验证集 80 张, 测试集 160 张
类别 City: 原始 800 张, 训练集 488 张, 验证集 80 张, 测试集 160 张
类别 Desert: 原始 800 张, 训练集 414 张, 验证集 80 张, 测试集 160 张
类别 Forest: 原始 800 张, 训练集 351 张, 验证集 80 张, 测试集 160 张
类别 Grassland: 原始 800 张, 训练集 298 张, 验证集 80 张, 测试集 160 张
类别 Highway: 原始 800 张, 训练集 252 张, 验证集 80 张, 测试集 160 张
类别 Lake: 原始 800 张, 训练集 214 张, 验证集 80 张, 测试集 160 张
类别 Mountain: 原始 800 张, 训练集 182 张, 验证集 80 张, 测试集 160 张
类别 Parking: 原始 800 张, 训练集 154 张, 验证集 80 张, 测试集 160 张
类别 Port: 原始 800 张, 训练集 131 张, 验证集 80 张, 测试集 160 张
类别 Railway: 原始 800 张, 训练集 111 张, 验证集 80 张, 测试集 160 张
类别 Residential: 原始 800 张, 训练集 94 张, 验证集 80 张, 测试集 160 张
类别 River: 原始 800 张, 训练集 80 张, 验证集 80 张, 测试集 160 张

划分后的类别数量统计：
Agriculture: 训练集 560 张, 验证集 80 张, 测试集 160 张
Airport: 训练集 560 张, 验证集 80 张, 测试集 160 张
Beach: 训练集 560 张, 验证集 80 张, 测试集 160 张
City: 训练集 488 张, 验证集 80 张, 测试集 160 张
Desert: 训练集 414 张, 验证集 80 张, 测试集 160 张


检查数据集图片数量函数

In [ ]:
import os
import glob

def count_images_in_folder(folder):
    # 定义常见的图片扩展名（均转为小写）
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    
    count = 0
    # 使用 os.walk 遍历文件夹及子文件夹
    for root, _, files in os.walk(folder):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext in image_extensions:
                count += 1
    return count

# 替换为你图片所在的文件夹路径
folder_path = 'path_to_your_image_folder'
num_images = count_images_in_folder(folder_path)
print("图片数量:", num_images)